***Train the model***

In [5]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

# Load the data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [6]:
# CIFAR10 has ten types of images labeled from 0 to 9. We only care about birds, which are labeled as class #2.
# So we'll cheat and change the Y values. Instead of each class being labeled from 0 to 9, we'll set it to True
# if it's a bird and False if it's not a bird.
y_train = (y_train ==2).astype(int)
y_test = (y_test == 2).astype(int)

# Normalize image data (pixel values from 0 to 255) to the 0-to-1 range
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

# Create a model and add layers
model = Sequential()

model.add(
    Conv2D(
        32, 
        (3,3), 
        padding="same", 
        input_shape=(32, 32, 3), 
        activation="relu"
    )
)
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=True
)

# Save the trained model to a file so we can use it to make predictions later
model.save("bird_model.h5")

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 207s 4ms/step - loss: 0.3218 - acc: 0.8916 - val_loss: 0.2870 - val_acc: 0.9020
Epoch 2/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.2498 - acc: 0.9064 - val_loss: 0.2278 - val_acc: 0.9102
Epoch 3/10
50000/50000 [==============================] - 240s 5ms/step - loss: 0.2321 - acc: 0.9121 - val_loss: 0.3188 - val_acc: 0.9056
Epoch 4/10
50000/50000 [==============================] - 249s 5ms/step - loss: 0.2197 - acc: 0.9170 - val_loss: 0.2208 - val_acc: 0.9155
Epoch 5/10
50000/50000 [==============================] - 246s 5ms/step - loss: 0.2053 - acc: 0.9221 - val_loss: 0.2810 - val_acc: 0.8888
Epoch 6/10
50000/50000 [==============================] - 247s 5ms/step - loss: 0.1934 - acc: 0.9266 - val_loss: 0.1881 - val_acc: 0.9273
Epoch 7/10
50000/50000 [==============================] - 250s 5ms/step - 

***Testing the network***

In [1]:
from keras.models import load_model
from keras.preprocessing import image
from pathlib import Path
import numpy as np

# Load the model we trained
model1 = load_model("bird_model.h5")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [6]:
for f in sorted(Path(".").glob("*.png")):
    
    # Load an image file to test
    image_to_test = image.load_img(str(f), target_size=(32, 32))
    
    # Convert the image data to a numpy array suitable for Keras 
    image_to_test = image.img_to_array(image_to_test)
    
    # Normalize the image the same way we normalized the training data (divide all the numbers by 255)
    image_to_test /= 255
    
    # Add a fourth dimension to the image since Keras expects a list of images 
    list_of_images = np.expand_dims(image_to_test, axis=0)
    
    # Make a prediction using the bird model
    results = model1.predict(list_of_images)
    
    # Since we only passed in one test image, we can just check the first result directly.
    image_likelihood = results[0][0]
    
    # The result will be a number from 0.0 to 1.0 representing the likelihood that this image is a bird.
    if image_likelihood > 0.5:
        print(f"{f} is most likely a bird! ({image_likelihood:.2f})")
    else:
        print(f"{f} is most likely NOT a bird! ({image_likelihood:.2f})")

bird1.png is most likely NOT a bird! (0.04)
bird2.png is most likely a bird! (0.90)
bird3.png is most likely NOT a bird! (0.28)
bird4.png is most likely NOT a bird! (0.46)
bird5.png is most likely NOT a bird! (0.30)
not_bird1.png is most likely NOT a bird! (0.00)
not_bird2.png is most likely NOT a bird! (0.01)
not_bird3.png is most likely NOT a bird! (0.02)
not_bird4.png is most likely NOT a bird! (0.00)
not_bird5.png is most likely NOT a bird! (0.00)


***Precision and recall***

In [8]:
from sklearn.metrics import confusion_matrix, classification_report
from keras.datasets import cifar10
from keras.models import load_model 

# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# CIFAR10 has ten types of images labeled from 0 to 9. We only care about birds, which are labeled as class #2.
# So we'll cheat and change the Y values. Instead of each class being labeled from 0 to 9, we'll set it to True
# if it's a bird and False if it's not a bird.
y_test = y_test == 2

# Normalize image data (pixel values from 0 to 255) to the 0-to-1 range
x_test = x_test.astype("float32")
x_test /= 255

# Load the model we trained 
model2 = load_model("bird_model.h5")
predictions = model2.predict(x_test, batch_size=32, verbose=1)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird"
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes 
tn, fp, fn, tp =  confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatvies: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)


10000/10000 [==============================] - 16s 2ms/step
True Positives: 342
True Negatives: 8948
False Positives: 52
False Negatvies: 658
             precision    recall  f1-score   support

      False       0.93      0.99      0.96      9000
       True       0.87      0.34      0.49      1000

avg / total       0.93      0.93      0.91     10000

